In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
#model_save_name = 'Copy_of_Kriti.pt'
#path = "/content/drive/MyDrive/Copy_of_Kriti2024.pt"
path = "/kaggle/input/for-prediction-kriti/Kriti2024_3rd_model.pt"
#model.load_state_dict(torch.load(path))
model = torch.load(path)
print(model)

OrderedDict([('l1.embeddings.word_embeddings.weight', tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0264],
        [-0.0132, -0.0673, -0.0161,  ..., -0.0227, -0.0554, -0.0260],
        [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0596, -0.0232],
        ...,
        [-0.0231, -0.0588, -0.0105,  ..., -0.0195, -0.0262, -0.0212],
        [-0.0490, -0.0561, -0.0047,  ..., -0.0107, -0.0180, -0.0219],
        [-0.0065, -0.0915, -0.0025,  ..., -0.0151, -0.0504,  0.0460]],
       device='cuda:0')), ('l1.embeddings.position_embeddings.weight', tensor([[ 0.0166, -0.0284, -0.0335,  ..., -0.0010, -0.0015,  0.0148],
        [ 0.0050,  0.0026, -0.0201,  ...,  0.0299,  0.0262, -0.0032],
        [-0.0140, -0.0054, -0.0127,  ...,  0.0162,  0.0170, -0.0070],
        ...,
        [ 0.0174,  0.0035, -0.0096,  ...,  0.0030,  0.0004, -0.0269],
        [ 0.0217, -0.0060,  0.0147,  ..., -0.0056, -0.0126, -0.0281],
        [ 0.0026, -0.0233,  0.0055,  ...,  0.0175,  0.0275, -0.0777]],
       

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd

In [3]:
test_dataset = pd.read_csv('/kaggle/input/for-prediction-kriti/test.csv')

In [4]:
from string import punctuation
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english') + list(punctuation)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import re
from nltk import word_tokenize
def preprocesing_text(text : str):

    try:
        text = text.lower()
        text = re.sub('[^a-zA-Z\s]', ' ', text)
        text = re.sub(r"(\s+|\t+|\r+|\n+)", " ",text)
        text = ' '.join([word for word in word_tokenize(text) if not word in stop_words])
        text = text.strip(' ')
        return text
    except Exception as e :
        print(e)
        return text

In [6]:
test_dataset.loc[:,'Abstract'] = test_dataset['Abstract'].apply(preprocesing_text)
test_dataset.loc[:,'Title'] = test_dataset['Title'].apply(preprocesing_text)

In [7]:
test_dataset.head(5)

,Id,Title,Abstract
0,30332,pricing fx options intermediate currency,suggest intermediate currency approach allows ...
1,50337,multicore processor based real time system aut...,paper propose intelligent management system ca...
2,66515,perceptual quality improvement videoconferenci...,latest years videoconferencing taken fundament...
3,57464,hundred kilobyte lookup tables efficient singl...,conventional super resolution sr schemes make ...
4,43169,efficient sequence labeling actor critic training,neural approaches sequence labeling often use ...


In [8]:
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=False,
    num_workers=0, drop_last=False,
    pin_memory=True,
)

In [9]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
class DistilBERTModel(torch.nn.Module):
    def __init__(self,n_classes):
        super(DistilBERTModel,self).__init__()
        self.l1 = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.linear = torch.nn.Linear(768,384)
        #self.linear = torch.nn.Linear(192,96)
        self.dropout = torch.nn.Dropout(0.2)
        self.final = torch.nn.Linear(768,n_classes)

    def forward(self,inp_id_title,att_mk_title,inp_id_abstract,att_mk_abstract):
        title_l1 = self.l1(inp_id_title,att_mk_title)
        abstract_l1 = self.l1(inp_id_abstract,att_mk_abstract)

        pooler_title = title_l1[0][:, 0]
        pooler_abstract = abstract_l1[0][:, 0]

        pooler_title = self.linear(pooler_title)
        pooler_title = torch.nn.ReLU()(pooler_title)

        pooler_abstract = self.linear(pooler_abstract)
        pooler_abstract = torch.nn.ReLU()(pooler_abstract)

        combined_features = torch.concat([pooler_title, pooler_abstract], dim=1)

        pooler = self.dropout(combined_features)
        output = self.final(pooler)
        # output = torch.nn.Sigmoid()(output)
        return output

In [11]:
path_f = "/kaggle/input/for-prediction-kriti/Kriti2024_3rd_model.pt"
state_dict = torch.load(path)

In [12]:
n_classes = 57

In [15]:
model = DistilBERTModel(n_classes)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [16]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBERTModel(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [18]:
model.eval()

DistilBERTModel(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [19]:
model.half()

DistilBERTModel(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [20]:
from tqdm import tqdm

In [21]:
class testsentiment:


  def __init__(self,test_dataset,tokenizer,model,max_len,params,threshold=0.5):
    self.titles=test_dataset['Title']
    self.abstracts =test_dataset['Abstract']
    self.tokenizer=tokenizer
    self.max_len=max_len

  def __getitem__(self,item):
    title = str(self.titles[item])
    abstract = str(self.abstracts[item])




    encoding_title = self.tokenizer.encode_plus(title,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            truncation=True,
                                            return_token_type_ids=False,
                                            padding='max_length',
                                            return_attention_mask=True,
                                            return_tensors='pt',)

    encoding_abstract = self.tokenizer.encode_plus(abstract,
                                               add_special_tokens=True,
                                               max_length=self.max_len,
                                               truncation=True,
                                               return_token_type_ids=False,
                                               padding='max_length',
                                               return_attention_mask=True,
                                               return_tensors='pt',)

    #title_ids = encoding_title['inp_id_title'].reshape(params['batch_size'],max_len).to(device,dtype=torch.long)
    #title_am = encoding_title['att_mk_title'].to(device,dtype=torch.long)

    #abstract_ids = encoding_abstract['inp_id_abstract'].reshape(params['batch_size'],max_len).to(device,dtype=torch.long)
    #abstract_am = encoding_abstract['att_mk_abstract'].to(device,dtype=torch.long)

    return {'inp_id_title':torch.as_tensor(encoding_title['input_ids'], dtype=torch.long),
               'att_mk_title':torch.as_tensor(encoding_title['attention_mask'], dtype=torch.long),
               'inp_id_abstract':torch.as_tensor(encoding_abstract['input_ids'], dtype=torch.long),
               'att_mk_abstract':torch.as_tensor(encoding_abstract['attention_mask'], dtype=torch.long)}

In [22]:
MAX_LEN = 512
PARAMS = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 0
          }

In [23]:
threshold=0.5

In [24]:
ts = testsentiment(test_dataset,tokenizer,model,MAX_LEN,PARAMS,threshold=0.5)

In [25]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [42]:
columns = ['Column_{}'.format(i) for i in range(57)]
df = pd.DataFrame(columns=columns)
df.shape
df

,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,...,Column_47,Column_48,Column_49,Column_50,Column_51,Column_52,Column_53,Column_54,Column_55,Column_56


In [43]:
for i in range(10974):
    item = ts.__getitem__(i)
    title_ids = item['inp_id_title']
    title_am = item['att_mk_title']
    abstract_ids = item['inp_id_abstract']
    abstract_am = item['att_mk_abstract']
    title_ids = title_ids.to(device)
    title_am = title_am.to(device)
    abstract_ids = abstract_ids.to(device)
    abstract_am = abstract_am.to(device)
    outputs = model(title_ids, title_am, abstract_ids, abstract_am)
    predictions = torch.as_tensor(outputs.data > threshold, dtype=torch.float)
    # Move predictions tensor to CPU and then convert to numpy array
    predictions_numpy = predictions.cpu().numpy()
    # Append predictions to DataFrame
    #df = df.append(pd.DataFrame(predictions_numpy, columns=target_columns), ignore_index=True)
    new_df = pd.DataFrame(predictions_numpy, columns=columns)

# Concatenate the existing DataFrame df with the new DataFrame new_df
    df = pd.concat([df, new_df], ignore_index=True)

/tmp/ipykernel_34/280147293.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


In [44]:
df.shape

(10974, 57)

In [45]:
df.head()

,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,...,Column_47,Column_48,Column_49,Column_50,Column_51,Column_52,Column_53,Column_54,Column_55,Column_56
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [46]:
df["Id"] = test_dataset["Id"]
df.shape

(10974, 58)

In [47]:
test_dataset.shape

(10974, 3)

In [39]:
df.shape

(21947, 58)

In [33]:
df.head()

,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,...,Column_48,Column_49,Column_50,Column_51,Column_52,Column_53,Column_54,Column_55,Column_56,Id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,30332.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50337.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,66515.0
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,57464.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,43169.0


In [48]:
df.shape

(10974, 58)

In [49]:
df.to_csv("submission_S.csv", index = False)